### Note
* Please do not view the notebook inside of github - there are compatibility issues. Downloading the Notebook should work.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

#Format 
def moneyFormat(num):
    return "${:20,.2f}".format(num)

def pctFormat(num):
     return '{:.1%}'.format(num)

## Player Count

* Display the total number of players


In [2]:
print('Total Player Count:', len(list(purchase_data['SN'].unique())))

Total Player Count: 576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [15]:

avgPrice = purchase_data['Price'].mean()
numUnique = len(list(purchase_data['Item ID'].unique()))
numCustomers = len(list(purchase_data['SN'].unique()))
df = pd.DataFrame({'Avg Price': [avgPrice], '# of Items': [numUnique], '# of Customers': [numCustomers]})
df['Avg Price'] = df['Avg Price'].apply(moneyFormat)
df

,Avg Price,# of Items,# of Customers
0,$ 3.05,179,576



## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [4]:
gender = purchase_data.groupby('Gender').count()

#Easier to just make a new column than use df.rename
gender['Count'] = gender['Price']  

#Remove all irrelavant columns
gender = pd.DataFrame(gender['Count']) 

#Express as Percentage
gender['Pct'] = gender['Count'] / sum(list(gender['Count'])) 
gender['Pct'] = gender['Pct'].apply(pctFormat)

gender

,Count,Pct
Gender,,
Female,113,14.5%
Male,652,83.6%
Other / Non-Disclosed,15,1.9%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [16]:
#Average Column
mean = pd.DataFrame(purchase_data.groupby('Gender').mean()['Price']).rename(columns={'Price': 'Avg Purchase Price'})

#Count Column
count = pd.DataFrame(purchase_data.groupby('Gender').count()['Price']).rename(columns={'Price': 'Purchase Count'})

#Merge
df = pd.merge(mean, count, left_index=True, right_index=True, how='inner')


#Effectively a multilayered groupby (which can be done explicitly, but this is easier in this case)
fAvg = float(purchase_data[purchase_data['Gender'] == 'Female'][['SN', 'Price']].groupby('SN').sum().mean())
mAvg = float(purchase_data[purchase_data['Gender'] == 'Male'][['SN', 'Price']].groupby('SN').sum().mean())
oAvg = float(purchase_data[purchase_data['Gender'] == 'Other / Non-Disclosed'][['SN', 'Price']].groupby('SN').sum().mean())
df['Avg Total per Person'] = [fAvg, mAvg, oAvg]

#Format
df['Avg Purchase Price'] = df['Avg Purchase Price'].apply(moneyFormat)
df['Avg Total per Person'] = df['Avg Total per Person'].apply(moneyFormat)

#Reorder
df = df[['Purchase Count', 'Avg Purchase Price', 'Avg Total per Person']]

df

,Purchase Count,Avg Purchase Price,Avg Total per Person
Gender,,,
Female,113,$ 3.20,$ 4.47
Male,652,$ 3.02,$ 4.07
Other / Non-Disclosed,15,$ 3.35,$ 4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [6]:
#Find upper and lower bound
minimum = purchase_data['Age'].min()
maximum = purchase_data['Age'].max()

#Generate bins
step = int((maximum - minimum) / 5)
bins = [x for x in range(minimum, maximum, step)]
bins.append(maximum + 1)

#Separrate Values
hist = pd.DataFrame(pd.cut(purchase_data['Age'], bins).value_counts())

#Add percentage column
hist['Pct'] = hist['Age'] / hist['Age'].sum()
hist['Pct'] = hist['Pct'].apply(pctFormat)

#Sort
hist.sort_index().rename(columns={'Age': 'Purchase Count'})



,Purchase Count,Pct
"(7, 14]",42,5.4%
"(14, 21]",297,38.5%
"(21, 28]",292,37.9%
"(28, 35]",100,13.0%
"(35, 42]",36,4.7%
"(42, 46]",4,0.5%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [7]:
#Separate into bins
binned = pd.DataFrame(pd.cut(purchase_data['Age'], bins))
binned.rename(columns={'Age': 'Age Group'}, inplace=True)

#Insert categorical column to main DataFrame
binned = pd.merge(purchase_data, binned, left_index=True, right_index= True).dropna()

#Generate avg Total per Player column
groups = list(binned['Age Group'].unique())
avgPerPerson = []
for group in groups:
    numPlayers = len(list(binned[binned['Age Group'] == group]['SN'].unique()))
    totalPurchased = binned[binned['Age Group'] == group]['Price'].sum()
    avgPerPerson.append(totalPurchased / numPlayers)

#Purchase Count Column
count = pd.DataFrame(binned.groupby('Age Group').count()['Item Name']).rename(columns={'Price': 'Count'})

#Avg Price Column
binned = pd.DataFrame(binned.groupby('Age Group').mean()['Price']).rename(columns={'Price': 'Avg Price'})

#Generate Columns
binned['Purchase Count'] = count
binned['Avg Total per Person'] = avgPerPerson
binned['Avg Total per Person'] = binned['Avg Total per Person'].apply(moneyFormat)
binned['Avg Price'] = binned['Avg Price'].apply(moneyFormat)

#Reorder Columns
binned = binned[['Purchase Count', 'Avg Price', 'Avg Total per Person']]

binned

,Purchase Count,Avg Price,Avg Total per Person
Age Group,,,
"(7, 14]",42,$ 3.02,$ 4.15
"(14, 21]",297,$ 3.06,$ 4.24
"(21, 28]",292,$ 3.00,$ 4.12
"(28, 35]",100,$ 3.01,$ 4.18
"(35, 42]",36,$ 3.41,$ 3.97
"(42, 46]",4,$ 2.77,$ 2.77


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [8]:
#Totals per Person
totals = pd.DataFrame(purchase_data.groupby('SN').sum()['Price']).sort_values(by = 'Price', ascending = False).rename(columns = {'Price': 'Total $ Spent'})

#Count per Person
counts = pd.DataFrame(purchase_data.groupby('SN').count()['Price']).rename(columns={'Price':'Purchase Count'})

#Merge
topSpenders = pd.merge(totals, counts, left_index=True, right_index=True, how='inner')

#Format
topSpenders['Total $ Spent'] = topSpenders['Total $ Spent'].apply(moneyFormat)

topSpenders.head(10)

,Total $ Spent,Purchase Count
SN,,
Lisosia93,$ 18.96,5
Idastidru52,$ 15.45,4
Chamjask73,$ 13.83,3
Iral74,$ 13.62,4
Iskadarya95,$ 13.10,3
Ilarin91,$ 12.70,3
Ialallo29,$ 11.84,3
Tyidaim51,$ 11.83,3
Lassilsala30,$ 11.51,3


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [18]:
items = purchase_data[['Item ID', 'Item Name', 'Price']]

#Total (sum)
total = pd.DataFrame(items.groupby(['Item ID', 'Item Name']).sum()).rename(columns = {'Price': 'Total Spent'})

#Count
counts = pd.DataFrame(items.groupby(['Item ID', 'Item Name']).count()).rename(columns = {'Price': 'Count'})
df = pd.merge(total, counts, left_index = True, right_index = True)

#Average
avg = pd.DataFrame(items.groupby(['Item ID', 'Item Name']).mean()).rename(columns = {'Price': 'Avg Price'})


#Master Merge
df = pd.merge(df, avg, left_index=True, right_index=True)

#Format
for col in ['Total Spent', 'Avg Price']:
    df[col] = df[col].apply(moneyFormat)
    
#Reorder
df = df[['Count', 'Total Spent', 'Avg Price']]
df.sort_values(by = 'Count', ascending = False)

,,Count,Total Spent,Avg Price
Item ID,Item Name,,,
92,Final Critic,13,$ 59.99,$ 4.61
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$ 50.76,$ 4.23
145,Fiery Glass Crusader,9,$ 41.22,$ 4.58
132,Persuasion,9,$ 28.99,$ 3.22
108,"Extraction, Quickblade Of Trembling Hands",9,$ 31.77,$ 3.53
...,...,...,...,...
42,The Decapitator,1,$ 1.75,$ 1.75
51,Endbringer,1,$ 4.66,$ 4.66
118,"Ghost Reaver, Longsword of Magic",1,$ 2.17,$ 2.17


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [10]:
df.sort_values(by = 'Total Spent', ascending = False).head(10)

# total

,,Count,Total Spent,Avg Price
Item ID,Item Name,,,
92,Final Critic,13,$ 59.99,$ 4.61
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$ 50.76,$ 4.23
82,Nirvana,9,$ 44.10,$ 4.90
145,Fiery Glass Crusader,9,$ 41.22,$ 4.58
103,Singed Scalpel,8,$ 34.80,$ 4.35
59,"Lightning, Etcher of the King",8,$ 33.84,$ 4.23
108,"Extraction, Quickblade Of Trembling Hands",9,$ 31.77,$ 3.53
78,"Glimmer, Ender of the Moon",7,$ 30.80,$ 4.40
72,Winter's Bite,8,$ 30.16,$ 3.77
